In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import sys
effdet_path = "../input/effdet"
sys.path.append(effdet_path)
timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
sys.path.append(timm_path)
import timm
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
omega_path = "../input/omegaconf"
sys.path.append(omega_path)
from omegaconf import OmegaConf
import glob
import sklearn
import math
import random

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torch import optim
from torchvision import transforms

from transformers import get_cosine_schedule_with_warmup

import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics, model_selection, preprocessing
from sklearn.model_selection import GroupKFold
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


In [ ]:
!pip install --no-deps '../input/pycocotools202/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl' > /dev/null

# Data Preparation

In [ ]:
df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")

In [ ]:
df = df[df.annotations != '[]']
df = df.reset_index(drop = True)

In [ ]:
df['fold'] = -1
kf = GroupKFold(n_splits = 5)
for fold, (train_idx, val_idx) in enumerate(kf.split(df, y = df.video_id.tolist(), groups=df.sequence)):
    df.loc[val_idx, 'fold'] = fold


In [ ]:
df.head()

In [ ]:
df.fold.value_counts()

In [ ]:
df['path'] = [f"../input/tensorflow-great-barrier-reef/train_images/video_{a}/{b}.jpg" for a,b in zip(df["video_id"],df["video_frame"])]
df['annotations'] = df['annotations'].apply(eval)
df.head()

In [ ]:
IMAGE_SIZE = 320

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches

def get_rectangle_edges_from_pascal_bbox(bbox):
    xmin_top_left, ymin_top_left, xmax_bottom_right, ymax_bottom_right = bbox

    bottom_left = (xmin_top_left, ymax_bottom_right)
    width = xmax_bottom_right - xmin_top_left
    height = ymin_top_left - ymax_bottom_right

    return bottom_left, width, height

def draw_pascal_voc_bboxes(
    plot_ax,
    bboxes,
    get_rectangle_corners_fn=get_rectangle_edges_from_pascal_bbox,
):
    for bbox in bboxes:
        bottom_left, width, height = get_rectangle_corners_fn(bbox)

        rect_1 = patches.Rectangle(
            bottom_left,
            width,
            height,
            linewidth=2,
            edgecolor="black",
            fill=False,
        )
        rect_2 = patches.Rectangle(
            bottom_left,
            width,
            height,
            linewidth=2,
            edgecolor="red",
            fill=False,
        )

        # Add the patch to the Axes
        plot_ax.add_patch(rect_1)
        plot_ax.add_patch(rect_2)

def draw_image(
    image, bboxes=None, draw_bboxes_fn=draw_pascal_voc_bboxes, figsize=(10, 10)
):
    fig, ax = plt.subplots(1, figsize=figsize)
    ax.imshow(image)

    if bboxes is not None:
        draw_bboxes_fn(ax, bboxes)

    plt.show()

In [ ]:
class DataAdaptor:
    def __init__(self,df):
        self.df = df
    def __len__(self):
        return len(self.df)
    
    def get_boxes(self, row):
        """Returns the bboxes for a given row as a 3D matrix with format [x_min, y_min, x_max, y_max]"""
        
        boxes = pd.DataFrame(row['annotations'], columns=['x', 'y', 'width', 'height']).astype(float).values
        
        # Change from [x_min, y_min, w, h] to [x_min, y_min, x_max, y_max]
        boxes[:, 2] = np.clip(boxes[:, 0] + boxes[:, 2],0,1280)
        boxes[:, 3] = np.clip(boxes[:, 1] + boxes[:, 3],0,720) 
        
        return boxes
    
    def get_image_bb(self , idx):
        img_src = self.df.loc[idx,'path']
        image   = cv2.imread(img_src)
        image   = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        row     = self.df.iloc[idx]
        bboxes  = self.get_boxes(row) 
        class_labels = np.ones(len(bboxes))
        return image, bboxes, class_labels, idx
    
        
    def show_image(self, index):
        image, bboxes, class_labels, image_id = self.get_image_bb(index)
        print(f"image_id: {image_id}")
        draw_image(image, bboxes.tolist())
        print(class_labels)     
        
        

In [ ]:
train_ds =DataAdaptor(df)

In [ ]:
im,bb,_,_ = train_ds.get_image_bb(4005)
bb

In [ ]:
train_ds.show_image(4005)

# Effdet Model

In [ ]:
from effdet import create_model

In [ ]:
# to know more what more pretrained models you can use, see here https://github.com/rwightman/efficientdet-pytorch/blob/9cb43186711d28bd41f82f132818c65663b33c1f/effdet/config/model_config.py
# 'tf_efficientdet_lite0' is one of the lightest model, you can use others

In [ ]:
model = create_model('tf_efficientdet_lite0' , bench_task='train' , num_classes= 1 , image_size=(IMAGE_SIZE,IMAGE_SIZE),bench_labeler=True,pretrained=True)

# Dataset & DataLoader

In [ ]:

def train_aug():
    return A.Compose(
        [
           
            A.Resize(IMAGE_SIZE,IMAGE_SIZE ,p = 1.0),
            A.Flip(0.5),  
            
       A.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
            ToTensorV2()
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format="pascal_voc", min_area=0, min_visibility=0, label_fields=["labels"]
        ),
    
    )


def val_aug():
    return  A.Compose(
        [ 
            A.Resize(IMAGE_SIZE,IMAGE_SIZE ,p = 1.0),
            A.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
            ToTensorV2()
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format="pascal_voc", min_area=0, min_visibility=0, label_fields=["labels"]
        ),
    )

In [ ]:
class CotsData(Dataset):
    def __init__(self , data_adaptor , transforms = None):
        self.ds = data_adaptor
        self.transforms = transforms
    
    def can_augment(self, boxes,image_size = 320): 
        box_outside_image = ((boxes[:, 0] < 0).any() or (boxes[:, 1] < 0).any() 
                             or (boxes[:, 2] > image_size).any() or (boxes[:, 3] > image_size).any())
        return not box_outside_image
    
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        image, bboxes, class_labels, image_id = self.ds.get_image_bb(idx)
        image = np.array(image, dtype=np.float32)
        sample = {
        "image": image,
        "bboxes": bboxes,
        "labels": class_labels,
                }
        sample = self.transforms(**sample)
        sample["bboxes"] = np.array(sample["bboxes"])
        image = sample["image"]
        bboxes = sample["bboxes"]
        labels = sample["labels"]
        _, new_h, new_w = image.shape
        sample["bboxes"][:, [0, 1, 2, 3]] = sample["bboxes"][:, [1, 0, 3, 2]]  # convert to yxyx

        target = {
            "bboxes": torch.as_tensor(sample["bboxes"], dtype=torch.float32),
            "labels": torch.as_tensor(labels),
            "image_id": torch.tensor([image_id]),
            "img_size": (new_h, new_w),
            "img_scale": torch.tensor([1.0]),}
        
        
        
        return image, target, image_id
     

In [ ]:
def collate_fn(batch):
        images, targets, image_ids = tuple(zip(*batch))
        images = torch.stack(images)
        images = images.float()

        boxes = [target["bboxes"].float() for target in targets]
        labels = [target["labels"].float() for target in targets]
        img_size = torch.tensor([target["img_size"] for target in targets]).float()
        img_scale = torch.tensor([target["img_scale"] for target in targets]).float()

        annotations = {
            "bbox": boxes,
            "cls": labels,
            "img_size": img_size,
            "img_scale": img_scale,
        }

        return images, annotations, targets, image_ids

# Training

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def train_one_epoch(train_loader,model,optimizer,e,epochs,scheduler):
    losses = AverageMeter()
    c_losses = AverageMeter()
    b_losses = AverageMeter()
    model.train()
    global_step = 0
    loop = tqdm(enumerate(train_loader),total = len(train_loader))
    
    for step,batch in loop:
        images, ann, _, image_ids = batch 
        batch_size = len(image_ids)
        images = images.to(device)
        target = {}
        target["bbox"] = [a.to(device) for a in ann['bbox']]
        target["cls"] = [a.to(device) for a in ann["cls"]]
        target["img_scale"] = (
            torch.tensor([1] * batch_size).float().to(device)
        )
        target["img_size"] = (
            torch.tensor( ann["img_size"]).to(device).float()
        )
        
        output = model(images , target)
        loss = output['loss']
        c_loss = output['class_loss']
        b_loss = output['box_loss']
     
        losses.update(loss.item(), batch_size)
        c_losses.update(c_loss.item(), batch_size)
        b_losses.update(b_loss.item(), batch_size)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        scheduler.step()
        global_step += 1
        
        loop.set_description(f"Epoch {e+1}/{epochs}")
        loop.set_postfix(loss = loss.item(), stage = 'train')
        
        
    return losses.avg, c_losses.avg , b_losses.avg

In [ ]:
def val_one_epoch(train_loader,model,e,epochs):
    losses = AverageMeter()
    c_losses = AverageMeter()
    b_losses = AverageMeter()
    model.eval()
    global_step = 0
    loop = tqdm(enumerate(train_loader),total = len(train_loader))
    
    for step,batch in loop:
        images, ann, _, image_ids = batch 
        batch_size = len(image_ids)
        images = images.to(device)
        target = {}
        target["bbox"] = [a.to(device) for a in ann['bbox']]
        target["cls"] = [a.to(device) for a in ann["cls"]]
        target["img_scale"] = (
            torch.tensor([1] * batch_size).float().to(device)
        )
        target["img_size"] = (
            torch.tensor( ann["img_size"]).to(device).float()
        )
         
        with torch.no_grad():
            output = model(images , target)
        loss = output['loss']
        c_loss = output['class_loss']
        b_loss = output['box_loss']
     
        losses.update(loss.item(), batch_size)
        c_losses.update(c_loss.item(), batch_size)
        b_losses.update(b_loss.item(), batch_size)
  

        global_step += 1
        
        loop.set_description(f"Epoch {e+1}/{epochs}")
        loop.set_postfix(loss = loss.item(), stage = 'val')
        
        
    return losses.avg, c_losses.avg , b_losses.avg

In [ ]:
def fit(m,fold_n ,train_bs=12, val_bs = 24):
    
    
    train_data= df[df.fold != fold_n]
    val_data  = df[df.fold == fold_n]
    
    train_ds =DataAdaptor(train_data.reset_index(drop=True))
    val_ds = DataAdaptor(val_data.reset_index(drop=True))
    
    train_data= CotsData(train_ds ,train_aug() )
    val_data  = CotsData(val_ds ,val_aug() )
    
    
    train_loader =DataLoader(
            train_data,
            batch_size=train_bs,
            shuffle=True,
            pin_memory=True,
            drop_last=True,
            num_workers=4,
            collate_fn=collate_fn,)
    
    valid_loader =DataLoader(
            val_data,
            batch_size=val_bs ,
            shuffle=False,
            drop_last=False,
            num_workers=4,
            collate_fn=collate_fn,)
   
    optimizer = optim.AdamW(m.parameters(), lr= 2e-4, weight_decay = 1e-6)
    epochs= 5
    warmup_epochs = 2
    num_train_steps = math.ceil(len(train_loader))
    num_warmup_steps= num_train_steps * warmup_epochs
    num_training_steps=int(num_train_steps * epochs)
    sch = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps = num_warmup_steps,num_training_steps =num_training_steps) 
    
    loop = range(epochs)
    for e in loop:
        
        total_loss,class_loss,box_loss = train_one_epoch(train_loader,m,optimizer,e,epochs,sch)
    
        print(f'For epoch {e+1}/{epochs}')
        print(f'average total_loss {total_loss}')
        print(f'average class_loss {class_loss}')
        print(f'average box_loss {box_loss}' )
        
        v_total_loss,v_class_loss,v_box_loss = val_one_epoch(valid_loader,m,e,epochs)
    
        print(f'For epoch {e+1}/{epochs}')
        print(f'average val total_loss {v_total_loss}')
        print(f'average val class_loss {v_class_loss}')
        print(f'average val box_loss {v_box_loss}' )
        
        torch.save(m.state_dict(),OUTPUT_DIR+ f'Fold {fold_n} model with val loss {v_total_loss}.pth') 

In [ ]:
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
model= model.to(device)
fit(model,0)

# References
1. https://medium.com/data-science-at-microsoft/training-efficientdet-on-custom-data-with-pytorch-lightning-using-an-efficientnetv2-backbone-1cdf3bd7921f
2. https://www.kaggle.com/shonenkov/training-efficientdet/notebook
3. https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-train-lb-0-416/notebook
4. https://github.com/benihime91/SIIM-COVID19-DETECTION-KAGGLE/blob/main/net-det/train.py
5. Most importantly thanks to this https://github.com/rwightman/efficientdet-pytorch

Infer coming soon(I guess)